In [45]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, fbeta_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score




X_train, X_test, y_train, y_test = transform_data(data)


In [46]:
import os
os.environ['MLFLOW_TRACKING_USERNAME']= "Abdellahi60"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "6870def210d626bd8f3c95f27c52207fb377ce80"

In [47]:
#setup mlflow
mlflow.set_tracking_uri('https://dagshub.com/Abdellahi60/mlops-bootcamp.mlflow') #your mlfow tracking uri
mlflow.set_experiment("IDSD_SD-experiment")

<Experiment: artifact_location='mlflow-artifacts:/17c91d3dd12c4d9faa80b11479c88b93', creation_time=1732651162195, experiment_id='0', last_update_time=1732651162195, lifecycle_stage='active', name='IDSD_SD-experiment', tags={}>

In [5]:
version = "v1.0"

In [48]:
data_url = "bac.csv"  # Mettez à jour le chemin vers votre fichier de données
bac = pd.read_csv(data_url)

In [50]:
bac['Decision'] = bac['Decision'].replace(['Ajourné', 'Abscent', 'Examen annulé à cause du Téléphone', 'Examen annulé à cause du comportement'], 'Ajourné')


In [51]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(bac, test_size=0.2, random_state=42)

In [53]:
train_data.head()

,Nom,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,moyeneGeneral,Decision,Note1,Note2,...,Note7,Note8,Willaya,moughataa,DateNaissance,ID,Today2021,Age,prenom,sexe
20967,Walad Haboub Nakh,SN,SN10,Las Palmas,College Teyarett 1,Ghabess Teyaret,3.539062,Ajourné,1.75,1.250,...,1.75,17.0,Nouakchott 1 (Nord),TEYARETT,2001-10-24,ID_22987,2021-11-25,20.0,Walad,F
10432,Saidou Abou Ba,SN,SN11,Rosso,Ecole Satara Rosso,Lycée Rosso,5.796875,Ajourné,5.25,4.750,...,1.00,16.0,Trarza,ROSSO,2000-01-17,ID_12169,2021-11-25,21.0,Saidou,M
20464,Djeinaba Malick Ba,SN,SN07,Zoueirat,Lycée Sebkha 1,EL AWLAMA SBK,8.453125,Sessionnaire,7.50,8.500,...,6.00,18.0,Nouakchott 2 (Ouest),SEBKHA,2002-04-21,ID_29121,2021-11-25,19.0,Djeinaba,F
16758,Ahmed Babe Ahmedou Abeidallah,SN,SN09,Toujounine,Lycée Jedida,CARAVANE DU SAVAOIR Ksar,7.296875,Ajourné,9.75,7.250,...,5.25,18.5,Nouakchott 2 (Ouest),KSAR,2003-07-09,ID_26568,2021-11-25,18.0,Ahmed,M
2718,Fatma Siyid Siyid,LM,LM4,Beileguet Litam,Ecole Sidi Abdoullah-Arafat,Lycée Arafat 1,5.705645,Ajourné,3.00,8.375,...,6.75,-1.0,Nouakchott 3 (Sud),ARAFAT,1999-01-06,ID_36182,2021-11-25,22.0,Fatma,F


In [54]:
train_data['Decision'].value_counts()

Ajourné         16325
Admis            1596
Sessionnaire     1194
Name: Decision, dtype: int64

In [55]:
# Split the data into features (X) and target (y)
X = train_data.drop(['Decision','moyeneGeneral', 'Note1','DateNaissance',
       'Note2', 'Note3', 'Note4', 'Note5', 'Note6', 'Note7', 'Note8'], axis=1)
y = pd.DataFrame({'Decision':train_data['Decision']})


In [56]:
X = X.set_index('ID')
y = y.set_index(pd.RangeIndex(start=0, stop=len(y)))

In [57]:
X.head()

,Nom,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,Willaya,moughataa,Today2021,Age,prenom,sexe
ID,,,,,,,,,,,,
ID_22987,Walad Haboub Nakh,SN,SN10,Las Palmas,College Teyarett 1,Ghabess Teyaret,Nouakchott 1 (Nord),TEYARETT,2021-11-25,20.0,Walad,F
ID_12169,Saidou Abou Ba,SN,SN11,Rosso,Ecole Satara Rosso,Lycée Rosso,Trarza,ROSSO,2021-11-25,21.0,Saidou,M
ID_29121,Djeinaba Malick Ba,SN,SN07,Zoueirat,Lycée Sebkha 1,EL AWLAMA SBK,Nouakchott 2 (Ouest),SEBKHA,2021-11-25,19.0,Djeinaba,F
ID_26568,Ahmed Babe Ahmedou Abeidallah,SN,SN09,Toujounine,Lycée Jedida,CARAVANE DU SAVAOIR Ksar,Nouakchott 2 (Ouest),KSAR,2021-11-25,18.0,Ahmed,M
ID_36182,Fatma Siyid Siyid,LM,LM4,Beileguet Litam,Ecole Sidi Abdoullah-Arafat,Lycée Arafat 1,Nouakchott 3 (Sud),ARAFAT,2021-11-25,22.0,Fatma,F


In [58]:
X.shape

(19115, 12)

In [59]:
# Split dataset into training set and test set
X_train = X.iloc[0:12930,:] 
X_test = X.iloc[12930:,:]
y_train = y.iloc[0:12930,:]
y_test =y.iloc[12930:,:]

In [60]:
X_train.reset_index(inplace=True)
X_test.reset_index(inplace=True)
y_train.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

In [61]:
# Sélection des caractéristiques (features) et la variable cible (target)
categorical_features = ['Etablissement', 'Serie,x', 'Centre', 'Willaya', 'moughataa']  
numeric_features = ['Age'] 
target = 'Decision' 

In [62]:
# Concaténation des ensembles d'entraînement et de test
all_data = pd.concat([X_train[categorical_features + numeric_features], X_test[categorical_features + numeric_features]], axis=0)

# Prétraitement des variables catégorielles
cat_transformer = OneHotEncoder(handle_unknown='ignore')
cat_transformer.fit(all_data[categorical_features])

# Prétraitement de la variable numérique
num_transformer = StandardScaler()
num_transformer.fit(all_data[numeric_features])

# Création du transformateur de colonnes
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, categorical_features),
        ('num', num_transformer, numeric_features)
    ])

# Encodage des variables catégorielles de l'ensemble d'entraînement
X_train_encoded = preprocessor.fit_transform(X_train[categorical_features + numeric_features])
X_test_encoded = preprocessor.transform(X_test[categorical_features + numeric_features])
y_train = y_train[target]
y_test = y_test[target]




In [22]:


# Supprimer les dernières 136 lignes
all_data1 = all_data.iloc[:-136]

# Vérifier si les lignes ont été supprimées
print(all_data.shape)


(19115, 6)


In [84]:
df3=all_data.head()
df3.head()

,Etablissement,"Serie,x",Centre,Willaya,moughataa,Age
0,Ghabess Teyaret,SN,College Teyarett 1,Nouakchott 1 (Nord),TEYARETT,20.0
1,Lycée Rosso,SN,Ecole Satara Rosso,Trarza,ROSSO,21.0
2,EL AWLAMA SBK,SN,Lycée Sebkha 1,Nouakchott 2 (Ouest),SEBKHA,19.0
3,CARAVANE DU SAVAOIR Ksar,SN,Lycée Jedida,Nouakchott 2 (Ouest),KSAR,18.0
4,Lycée Arafat 1,LM,Ecole Sidi Abdoullah-Arafat,Nouakchott 3 (Sud),ARAFAT,22.0


In [85]:
# Sauvegarder le DataFrame dans un fichier CSV
df3.to_csv('test_data.csv', index=False)

In [17]:
pip install imbalanced-learn


In [63]:
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import NearestNeighbors

# Créez une instance du modèle de voisinage
nn = NearestNeighbors(n_neighbors=3)

# Ajustez le modèle de voisinage sur vos données
nn.fit(X_train_encoded)

# Créez une instance de la classe SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=nn)

# Rééchantillonnez votre ensemble de données à l'aide de SMOTE
X_resampled, y_resampled = smote.fit_resample(X_train_encoded, y_train)


In [22]:
mlflow.sklearn.autolog(disable=True)

In [23]:
with mlflow.start_run(run_name='LogisticRegression'):
    mlflow.log_param("data_url",data_url)
    mlflow.log_param("input_rows",bac.shape[0])
    mlflow.log_param("input_cols",bac.shape[1])
    #model fitting and training
    lr=LogisticRegression(max_iter=1000)
    mlflow.set_tag(key= "model",value="LogisticRegression")
    params = lr.get_params()
    mlflow.log_params(params)
    lr.fit(X_resampled,y_resampled)
    train_features_name = f'{X_resampled=}'.split('=')[0]
    train_label_name = f'{y_resampled=}'.split('=')[0]
    mlflow.set_tag(key="train_features_name",value= train_features_name)
    mlflow.set_tag(key= "train_label_name",value=train_label_name)
    predicted=lr.predict(X_test_encoded)
    precision,recall,fscore,support=score(y_test,predicted,average='macro')
    mlflow.log_metric("Precision_test",precision)
    mlflow.log_metric("Recall_test",recall)
    mlflow.log_metric("F1_score_test",fscore)
    mlflow.sklearn.log_model(lr,artifact_path="ML_models")


2024/11/27 23:19:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LogisticRegression at: https://dagshub.com/Abdellahi60/mlops-bootcamp.mlflow/#/experiments/0/runs/21ec283e2bda4439b65688008cb89f7c
🧪 View experiment at: https://dagshub.com/Abdellahi60/mlops-bootcamp.mlflow/#/experiments/0


In [24]:
lr=LogisticRegression(max_iter=1000)
lr.fit(X_resampled,y_resampled)
predicted=lr.predict(X_test_encoded)

##  RandomForestClassifier

In [64]:
from sklearn.preprocessing import LabelEncoder

# Créez une instance de LabelEncoder
label_encoder = LabelEncoder()

# Appliquez le LabelEncoder aux classes de la variable cible
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


#### XGBOOST Model

In [79]:
import xgboost as xgb
from xgboost import XGBClassifier
import mlflow
import mlflow.xgboost
from sklearn.metrics import f1_score, fbeta_score

# Create an XGBClassifier with your desired hyperparameters
xgb_model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.2,
    gamma=0.3,
    booster='gblinear'
)

# Start a new MLflow run
with mlflow.start_run(run_name='XGBoost'):
    # Train the model
    xgb_model.fit(X_train_encoded, y_train_encoded)

    # Make predictions
    y_pred = xgb_model.predict(X_test_encoded)

    # Calculate F1-score
    f1 = f1_score(y_test_encoded, y_pred, average='micro')

    # Calculate F2-score
    f2 = fbeta_score(y_test_encoded, y_pred, beta=2, average='micro')

    # Log XGBoost parameters
    mlflow.log_params(xgb_model.get_params())

    # Log F1-score as a metric
    mlflow.log_metric("F1_score_test", f1)

    # Log F2-score as a metric
    mlflow.log_metric("F2-score", f2)
    
    # Log the XGBoost model
    mlflow.xgboost.log_model(xgb_model,artifact_path="ML_models")


C:\Users\TOSHIBA\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [03:02:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "gamma" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\TOSHIBA\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [03:02:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
2024/12/05 03:03:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBoost at: https://dagshub.com/Abdellahi60/mlops-bootcamp.mlflow/#/experiments/0/runs/86778d570cf54359818dee0557c2a489
🧪 View experiment at: https://dagshub.com/Abdellahi60/mlops-bootcamp.mlflow/#/experiments/0


In [ ]:
# Vous pouvez enregistrer les données d'entraînement et de test si nécessaire
#train_data.to_csv("train_set.csv", index=False)
#test_data.to_csv("test_set.csv", index=False)


In [81]:
#Reading Pandas Dataframe from mlflow
all_experiments = [exp.experiment_id for exp in mlflow.search_experiments()]
df_mlflow = mlflow.search_runs(experiment_ids=all_experiments,filter_string="metrics.F1_score_test <1")
run_id = df_mlflow.loc[df_mlflow['metrics.F1_score_test'].idxmax()]['run_id']
print(run_id)

86778d570cf54359818dee0557c2a489


In [32]:
df_mlflow

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.F2-score,metrics.F1_score_test,metrics.Recall_test,metrics.Precision_test,...,params.l1_ratio,params.solver,tags.mlflow.source.type,tags.mlflow.log-model.history,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.name,tags.train_features_name,tags.train_label_name,tags.model
0,3fc9b373ab44498fac7b1bc27020f9f2,0,FINISHED,mlflow-artifacts:/17c91d3dd12c4d9faa80b11479c8...,2024-11-28 18:05:39.619000+00:00,2024-11-28 18:06:24.088000+00:00,0.797251,0.797251,NaN,NaN,...,None,None,LOCAL,"[{""run_id"": ""3fc9b373ab44498fac7b1bc27020f9f2""...",XGBoost,abdellahi60,C:\Users\TOSHIBA\anaconda3\lib\site-packages\i...,None,None,None
1,6d2fef937fbe49f0a537876c196579c9,0,FAILED,mlflow-artifacts:/17c91d3dd12c4d9faa80b11479c8...,2024-11-27 23:29:58.249000+00:00,2024-11-28 18:02:00.447000+00:00,NaN,0.229810,0.246312,0.222875,...,None,None,LOCAL,None,RandomForestClassifier,abdellahi60,C:\Users\TOSHIBA\anaconda3\lib\site-packages\i...,X_resampled,y_resampled,RandomForestClassifier
2,21ec283e2bda4439b65688008cb89f7c,0,FINISHED,mlflow-artifacts:/17c91d3dd12c4d9faa80b11479c8...,2024-11-27 23:15:55.657000+00:00,2024-11-27 23:20:10.560000+00:00,NaN,0.185616,0.316496,0.222966,...,None,lbfgs,LOCAL,"[{""run_id"": ""21ec283e2bda4439b65688008cb89f7c""...",LogisticRegression,abdellahi60,C:\Users\TOSHIBA\anaconda3\lib\site-packages\i...,X_resampled,y_resampled,LogisticRegression
3,24d75e26c0004d9d8fa9b805ff8e86cb,0,FINISHED,mlflow-artifacts:/17c91d3dd12c4d9faa80b11479c8...,2024-11-27 22:32:39.362000+00:00,2024-11-27 22:33:23.206000+00:00,NaN,0.184899,0.315499,0.222108,...,None,lbfgs,LOCAL,"[{""run_id"": ""24d75e26c0004d9d8fa9b805ff8e86cb""...",LogisticRegression,abdellahi60,C:\Users\TOSHIBA\anaconda3\lib\site-packages\i...,X_resampled,y_resampled,LogisticRegression


In [82]:
#let's call the model from the model registry ( in production stage)
import mlflow.pyfunc

logged_model = f'runs:/{run_id}/ML_models'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
print(loaded_model)

# Predict on a Pandas DataFrame.

loaded_model.predict(X_test_encoded)

mlflow.pyfunc.loaded_model:
  artifact_path: ML_models
  flavor: mlflow.xgboost
  run_id: 86778d570cf54359818dee0557c2a489



array([1, 1, 1, ..., 1, 1, 1], dtype=int64)